In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import json

In [7]:
# Enable mixed precision and XLA JIT compilation
tf.keras.mixed_precision.set_global_policy('mixed_float16')
tf.config.optimizer.set_jit(True)

# Load data (use smaller sequences by downsampling for speed)
print("Loading datasets...")
X_train = np.load('X_train.npy')[:, ::2, :]  # Downsample by 2
y_train = np.load('y_train.npy').astype(np.int32)  # Ensure sparse labels
X_val = np.load('X_val.npy')[:, ::2, :]
y_val = np.load('y_val.npy').astype(np.int32)
X_test = np.load('X_test.npy')[:, ::2, :]
y_test = np.load('y_test.npy').astype(np.int32)

# Print dataset shapes
print(f"Train set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")

Loading datasets...
Train set: (19524, 12, 32), (19524,)
Validation set: (2441, 12, 32), (2441,)
Test set: (2440, 12, 32), (2440,)


In [8]:
# Build a blazing-fast, lightweight LSTM model
print("\nBuilding the Ultra-Light LSTM model...")
model = Sequential([
    LSTM(1, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False),
    BatchNormalization(),  # Stabilizes training
    Dense(1, activation='sigmoid', dtype='float32')  # Ensure output remains in float32
])

# Compile the model
print("\nCompiling the model...")
model.compile(optimizer=Adam(learning_rate=0.005), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with early stopping
print("\nTraining the model...")
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history = model.fit(
    X_train,
    y_train,
    epochs=15,  # Extremely fast training
    batch_size=512,  # Large batch size for speed
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
    verbose=1
)

print("Model training completed.")


Building the Ultra-Light LSTM model...

Compiling the model...

Training the model...
Epoch 1/15


C:\Users\ebarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.5590 - loss: 0.7549 - val_accuracy: 0.8755 - val_loss: 0.5531
Epoch 2/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7604 - loss: 0.5484 - val_accuracy: 0.9394 - val_loss: 0.4582
Epoch 3/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9198 - loss: 0.4191 - val_accuracy: 0.9385 - val_loss: 0.3689
Epoch 4/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9543 - loss: 0.3135 - val_accuracy: 0.9443 - val_loss: 0.2932
Epoch 5/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9590 - loss: 0.2429 - val_accuracy: 0.9492 - val_loss: 0.2369
Epoch 6/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9581 - loss: 0.2036 - val_accuracy: 0.9545 - val_loss: 0.1997
Epoch 7/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9631 - loss: 0.1725 - val_accuracy: 0.9599 - val_loss: 0.1760
Epoch 8/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9584 - loss: 0.1694 - val_accuracy: 0.9615 - val_loss: 0.1

In [9]:
# Save the trained model
print("\nSaving the trained model to 'ultra_light_lstm_model.h5'...")
model.save('ultra_light_lstm_model.h5')
print("Model saved successfully.")

# Evaluate the model on test data
print("\nEvaluating the model on test data...")
y_pred_test = (model.predict(X_test) > 0.5).astype(int)
test_metrics = {
    "accuracy": (y_pred_test == y_test).mean(),
    "precision": precision_score(y_test, y_pred_test),
    "recall": recall_score(y_test, y_pred_test),
    "f1_score": f1_score(y_test, y_pred_test),
    "confusion_matrix": confusion_matrix(y_test, y_pred_test).tolist(),
}
print("Test data metrics computed.")

# Summarize results
print("\nFinal Training Loss and Validation Loss")
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
print(f"Final Training Loss: {final_train_loss:.4f}, Validation Loss: {final_val_loss:.4f}")


Saving the trained model to 'ultra_light_lstm_model.h5'...
Model saved successfully.

Evaluating the model on test data...
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Test data metrics computed.

Final Training Loss and Validation Loss
Final Training Loss: 0.1409, Validation Loss: 0.1374


In [10]:
# Create a Model Card
print("\nCreating Model Card...")
model_card = {
    "Model Name": "Ultra-Lightweight LSTM",
    "Version": "1.2",
    "Model Architecture": "Single LSTM Unit, BatchNorm, Dense Sigmoid Output",
    "Purpose": "Predict sepsis risk using time-series health data.",
    "Dataset": {
        "Train Shape": X_train.shape,
        "Validation Shape": X_val.shape,
        "Test Shape": X_test.shape,
    },
    "Training Configuration": {
        "Epochs": len(history.history['loss']),
        "Batch Size": 512,
        "Optimizer": "Adam",
        "Learning Rate": 0.005,
        "Loss Function": "Binary Crossentropy",
    },
    "Performance Metrics (Test Data)": test_metrics,
    "Limitations": [
        "Simplified architecture may underperform on more complex datasets.",
        "Requires further validation on diverse patient datasets.",
    ],
    "Ethical Considerations": [
        "Validate predictions with human oversight.",
        "Ensure fairness across patient demographics.",
    ],
    "Author": "Eric Barnes",
    "Date": "2024-11-18",
}
print("Model Card created.")

# Save the Model Card
print("\nSaving Model Card to 'ultra_light_model_card.json'...")
with open('ultra_light_model_card.json', 'w') as f:
    json.dump(model_card, f, indent=4)
print("Model Card saved successfully.")


Creating Model Card...
Model Card created.

Saving Model Card to 'ultra_light_model_card.json'...
Model Card saved successfully.
